In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'D:\MLPROJECTS\Language_Recognition\dataset.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

So i will be using spaCy model for the purpose...
steps:
1. load the universal spaCy model
2. prepare the data
3. preprocess the data (by cleaning it i.e. removing the multiple spaces if present)
4. training data processing (splitting the data)
5. Then i will be doing the vectorization
6. After this i will decide what i want to do, since there are multiple classifiers that i can use after Vectorization
Maybe i will follow something like the thing that i did in my sentiment analysis project

In [29]:
nlp = spacy.load("xx_ent_wiki_sm")

In [30]:
df = pd.read_csv(r'D:\MLPROJECTS\Language_Recognition\dataset.csv')
print(df.head())

                                                Text  language
0  klement gottwaldi surnukeha palsameeriti ning ...  Estonian
1  sebes joseph pereira thomas  på eng the jesuit...   Swedish
2  ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...      Thai
3  விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...     Tamil
4  de spons behoort tot het geslacht haliclona en...     Dutch


**Lets begin our preprocessing step**

In [31]:
# import re

def yele(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

df['Text'] = df['Text'].apply(yele)

In [32]:
print(df.shape)
# print(df.iloc[10334][0])

(22000, 2)


***Lets begin our training data processing***

In [33]:
X = df['Text']
y = df['language']

X_t, X_test, y_t, y_test = train_test_split(X,y,test_size = 0.1, random_state = 1)

print(len(X_t))
print(len(X_test))

19800
2200


Now the vectorization process begins from here:

In [34]:
vect = CountVectorizer()
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y_t_vec = enc.fit_transform(y_t)
y_test_vec = enc.transform(y_test)
X_t_vec = vect.fit_transform(X_t)
X_test_vec = vect.transform(X_test)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_t_tf = tfidf.fit_transform(X_t)
X_test_tf = tfidf.transform(X_test)

print(f"TF-IDF training data shape: {X_t_tf.shape}")
print(f"TF-IDF testing data shape: {X_test_tf.shape}")

TF-IDF training data shape: (19800, 5000)
TF-IDF testing data shape: (2200, 5000)


In [36]:
from sklearn.naive_bayes import MultinomialNB
clf_nb = MultinomialNB()
clf_nb.fit(X_t_tf, y_t_vec)
predi = clf_nb.predict(X_test_tf)
accuracy_logreg = accuracy_score(y_test_vec, predi)
print(f"Logistic Regression Accuracy: {accuracy_logreg * 100:.2f}%")

Logistic Regression Accuracy: 91.41%


In [37]:
from sklearn.linear_model import LogisticRegression
logi = LogisticRegression(max_iter = 1000)

logi.fit(X_t_tf, y_t_vec)

predi = logi.predict(X_test_tf)

accuracy_logreg = accuracy_score(y_test_vec, predi)
print(f"Logistic Regression Accuracy: {accuracy_logreg * 100:.2f}%")


Logistic Regression Accuracy: 93.68%


In [38]:
yele = ['Hello my name is this', 'salut, je mappelle ainsi' , 'привет, меня зовут это', ]
yele_tf = tfidf.transform(yele)

answer = logi.predict(yele_tf)
ans = enc.inverse_transform(answer)
print(ans)

['Japanese' 'Chinese' 'Chinese']


In [39]:
from sklearn.svm import SVC
clf_svm = SVC(kernel='linear', max_iter=1000)
clf_svm.fit(X_t_tf, y_t_vec)
predi = clf_svm.predict(X_test_tf)
accuracy_logreg = accuracy_score(y_test_vec, predi)
print(f"Logistic Regression Accuracy: {accuracy_logreg * 100:.2f}%")

c:\Users\jiten\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Logistic Regression Accuracy: 93.64%


In [40]:
import pickle

model_path = r'D:\MLPROJECTS\Language_Recognition\language_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(clf_svm, f)


In [41]:
import pandas as pd
import spacy
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder

# Load multilingual spaCy model
nlp = spacy.load("xx_ent_wiki_sm")

# Load dataset
df = pd.read_csv("dataset.csv")  # Make sure it's in the same folder or adjust path

# Clean text
def clean_text(text):
    doc = nlp(str(text))
    return " ".join([token.text for token in doc if not token.is_stop and not token.is_punct])

df['Text'] = df['Text'].apply(clean_text)

# Prepare features and labels
X = df['Text']
y = df['language']

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Vectorize text
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

# Train model
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Save model, vectorizer, and encoder
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("Model, vectorizer, and encoder saved!")


Model, vectorizer, and encoder saved!
